In [ ]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null  
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz  
!pip install -q findspark
!pip install pycountry
!pip install isbnlib

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:13 https://developer.downlo

In [ ]:
import findspark
findspark.init("spark-3.3.1-bin-hadoop3")
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql import Row
import pyspark.sql.functions as f
from pyspark.sql.functions import format_number
from pyspark.sql.functions import col, split, when
import pandas as pd
spark = SparkSession.builder.appName("Basic").getOrCreate()
import requests as req
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import ltrim
from pyspark.sql.functions import *
import pycountry
from isbnlib import is_isbn10, is_isbn13
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col,isnan,when,count
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window
from pyspark.sql.functions import trim
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
Users = spark.read.option("header", True).csv('/content/drive/MyDrive/Books_Rating/DataBase/BX-Users.csv')
Users.show(truncate = False)

In [ ]:
Users = Users.withColumnRenamed('"User-ID;""Location"";""Age"""', 'User-ID')
Users = Users.withColumn('User-ID', regexp_replace('User-ID', '"', ''))

In [ ]:
Users = Users.withColumn('UserID', split(Users['User-ID'], ';').getItem(0)) \
       .withColumn('Location', split(Users['User-ID'], ';').getItem(1)) \
       .withColumn('Age', split(Users['User-ID'], ';').getItem(2))
Users = Users.drop("User-ID", '"').withColumn("Age",col("Age").cast("int"))
Users = Users.na.fill(value=0,subset=["Age"])
Users = Users.filter(Users['Location'].isNotNull())
Users = Users.withColumn("last_word", f.element_at(f.split("location"," "),-1))

In [ ]:
rows = Users.select('Location').collect()

countries = list(pycountry.countries)
countries
c = []
for country in countries:
  c.append(country.name)

In [ ]:
DistinctList = Users.select(f.collect_set('last_word')).collect()[0][0]

Users = Users.replace(['kingdom', 'usa', 'Zealand', 'Russia', 'Taiwan', 'Arabia', 'Vietnam', 'Iran'], \
                      ['united kingdom', 'united states', 'New Zealand', 'Russian Federation', 'Taiwan, Province of China', 'Saudi Arabia', 'Viet Nam', 'Iran, Islamic Republic of'], 'last_word')
Users = Users.withColumn("UserID",ltrim(col("UserID"))).withColumn("Age",ltrim(col("Age"))).withColumn("last_word",ltrim(col("last_word"))) \
        .withColumn('last_word', regexp_replace('last_word', "None", 'Unknown')).withColumn("last_word", when(col("last_word")=="" ,'Unknown').otherwise(col("last_word")))
        
Users.show(truncate = False)
print((Users.count(), len(Users.columns)))

+------+------------------------------------+---+--------------+
|UserID|Location                            |Age|last_word     |
+------+------------------------------------+---+--------------+
|1     |nyc new york usa                    |0  |united states |
|2     |stockton california usa             |18 |united states |
|3     |moscow yukon territory russia       |0  |russia        |
|4     |porto v.n.gaia portugal             |17 |portugal      |
|5     |farnborough hants united kingdom    |0  |united kingdom|
|6     |santa monica california usa         |61 |united states |
|7     |washington dc usa                   |0  |united states |
|8     |timmins ontario canada              |0  |canada        |
|9     |germantown tennessee usa            |0  |united states |
|10    |albacete wisconsin spain            |26 |spain         |
|11    |melbourne victoria australia        |14 |australia     |
|12    |fort bragg california usa           |0  |united states |
|13    |barcelona barcelo

In [ ]:
Users = Users.withColumn("last_word", initcap(col('last_word')))
Users = Users.withColumn("last_word", when(col("last_word").isin(c), Users.last_word).otherwise("Unknown"))
Users = Users.drop('Location').withColumnRenamed('last_word', 'Country')

Users.show(truncate = False)
print((Users.count(), len(Users.columns)))

+------+---+--------------+
|UserID|Age|Country       |
+------+---+--------------+
|1     |0  |United States |
|2     |18 |United States |
|3     |0  |Unknown       |
|4     |17 |Portugal      |
|5     |0  |United Kingdom|
|6     |61 |United States |
|7     |0  |United States |
|8     |0  |Canada        |
|9     |0  |United States |
|10    |26 |Spain         |
|11    |14 |Australia     |
|12    |0  |United States |
|13    |26 |Spain         |
|14    |0  |United States |
|15    |0  |Canada        |
|16    |0  |United States |
|17    |0  |United States |
|18    |25 |Brazil        |
|19    |14 |Unknown       |
|20    |19 |United States |
+------+---+--------------+
only showing top 20 rows

(278859, 3)


In [ ]:
Users = Users.filter(col("UserID") != 'unknown') 
Users = Users.withColumnRenamed('UserID', 'UserBK')
Users = Users.withColumn("UserSK", row_number().over(Window.orderBy(monotonically_increasing_id()))+100)
Users = Users.select('UserSK', 'UserBK', 'Country', 'Age')

DIM_USERS = Users
DIM_USERS.show()
DIM_USERS.describe().show()  

+------+------+--------------+---+
|UserSK|UserBK|       Country|Age|
+------+------+--------------+---+
|   101|     1| United States|  0|
|   102|     2| United States| 18|
|   103|     3|       Unknown|  0|
|   104|     4|      Portugal| 17|
|   105|     5|United Kingdom|  0|
|   106|     6| United States| 61|
|   107|     7| United States|  0|
|   108|     8|        Canada|  0|
|   109|     9| United States|  0|
|   110|    10|         Spain| 26|
|   111|    11|     Australia| 14|
|   112|    12| United States|  0|
|   113|    13|         Spain| 26|
|   114|    14| United States|  0|
|   115|    15|        Canada|  0|
|   116|    16| United States|  0|
|   117|    17| United States|  0|
|   118|    18|        Brazil| 25|
|   119|    19|       Unknown| 14|
|   120|    20| United States| 19|
+------+------+--------------+---+
only showing top 20 rows

+-------+-----------------+-----------------+-----------+------------------+
|summary|           UserSK|           UserBK|    Country|

In [ ]:
DIM_USERS.createTempView("Users")

spark.sql(
    '''
    select count(Country) as Unknown from Users
    where Country = 'Unknown'
    '''
).show()

+-------+
|Unknown|
+-------+
|  11509|
+-------+



In [ ]:
pandasDF = DIM_USERS.toPandas()
pandasDF.to_csv(r'/content/drive/MyDrive/Books_Rating/Fixed_CSV/DIM_USERS.csv',mode='w+', encoding='utf-8', index=False)

In [ ]:
spark.stop()